In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys
import spotipy
import spotipy.util as util
import json
import time

import pandas as pd

In [3]:
with open('configs.secret') as f:
    data = json.load(f)
    
#scope = 'user-library-read'
scope = None
username = data['username']
client_id = data['client_id']
client_secret = data['client_secret']
redirect_uri = data['redirect_uri']

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [4]:
if not token:
    print("Can't get token for", username)
    
sp = spotipy.Spotify(auth=token)

In [10]:
playlists = []
num_playlists = 10
limit = 50

for i in range (0, int(num_playlists / limit)):
    results = sp.search('Banger', limit=limit, offset=0, type='playlist')
    playlists.extend(results['playlists']['items'])
    # print(len(playlists))
    time.sleep(0.1)


In [12]:
print(len(playlists))
tracks = []
next = True

while (next):
    results = sp.playlist_tracks(playlists[1]['id'])
    next = results['next']
    tracks.extend(results[tracks])



0


IndexError: list index out of range

In [ ]:
# artist.hotttnesss,artist.id,artist.name,artist_mbtags,artist_mbtags_count,bars_confidence,bars_start,beats_confidence,beats_start,duration,end_of_fade_in,familiarity,key,key_confidence,latitude,location,longitude,loudness,mode,mode_confidence,release.id,release.name,similar,song.hotttnesss,song.id,start_of_fade_out,tatums_confidence,tatums_start,tempo,terms,terms_freq,time_signature,time_signature_confidence,title,year
